In [1]:
! pip install openai tiktoken langchain gradio


[notice] A new release of pip is available: 23.1.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import gradio as gr
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
from elevenlabs import voices, generate, play, save

eleven_api_key = "34dbe78c2a8c6642fcdc27dbb44c1929"

In [3]:
def summarize_and_sonify(file, num_minutes, voice):

    # Read doc
    doc = PyPDFLoader(file.name).load()

    # Create string from doc content
    text = ''
    for page in doc:
        text += page.page_content

    num_words = num_minutes*135 # Approximately 135 words per minute with TTS voice (refine this)

    audio_voice = voice.split()[0]
    summary_lang = "English"
    if voice.split()[1] == "(SWE)":
        summary_lang = "Swedish"
    if voice.split()[1] == "(IT)":
        summary_lang = "Italian"

    # Connect to OpenAI, prompt ChatGPT to summarize the text
    chat_model = ChatOpenAI() # Must have set API key as env var
    prompt = "Summarize the following text in {} words. The summary must be in {}. Text: {}".format(num_words, summary_lang, text) 
    summary = chat_model.predict(prompt)

    # Connect to elevenlabs, generate audio
    audio = generate(text=summary, voice=audio_voice, model="eleven_monolingual_v1", api_key=eleven_api_key)

    # Save audiofile
    if not os.path.exists("audiofiles"):
        os.makedirs("audiofiles")

    # Handling identical file names
    file_suffix = 0
    while os.path.exists("audiofiles/summary_in_{}_mins_{}.wav".format(num_minutes, file_suffix)):
        file_suffix +=1

    save(audio, "audiofiles/summary_in_{}_mins_{}.wav".format(num_minutes, file_suffix))

    return "audiofiles/summary_in_{}_mins_{}.wav".format(num_minutes, file_suffix)




In [6]:
demo = gr.Interface(fn=summarize_and_sonify, 
                    inputs=[
                            "file", 
                            gr.Slider(minimum=0.5, maximum=5, step=0.5, label="Length", info = "Number of minutes of audio summary"),
                            gr.Dropdown(["Adam (US)", "Antoni (IT)", "Arnold (US)", "Bella (US)", "Domi (US)", "Elli (US)", "Josh (SWE)", "Rachel (US)", "Sam (US)"], label="Voice", info="Narrator voice and language of audio summary"),
                            ],
                    outputs= ["audio"]
                    )
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://f988e1ca2014ce7bf2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [5]:
# voices = voices()
# for i in range(30):
#     print(voices[i].name)